<a href="https://colab.research.google.com/github/iwaohig/linworks-oauth-colab/blob/main/LINEWORKSOauth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # 更新
!apt install chromium-chromedriver # ChromeDriverのインストール
!cp /usr/lib/chromium-browser/chromedriver /usr/bin # パスへのコピー
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') # Pythonのパスに追加

In [9]:
CLIENT_ID = '' #@param {type:"string"}
CLIENT_SECRET = '' #@param {type:"string"}
REDIRECT_URI = '' #@param {type:"string"}
SCOPE = '' #@param {type:"string"}
STATE = '' #@param {type:"string"}
LOGIN_ID = '' #@param {type:"string"}
PASSWORD = '' #@param {type:"string"}

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoAlertPresentException
import sys
import time
import requests
import urllib.parse

# OAuth認証のためのURL構築
AUTH_BASE_URL = 'https://auth.worksmobile.com/oauth2/v2.0/authorize'
auth_params = {
    'client_id': CLIENT_ID,
    'redirect_uri': REDIRECT_URI,
    'scope': SCOPE,
    'response_type': 'code',
    'state': STATE
}
auth_query = urllib.parse.urlencode(auth_params)
auth_url = f'{AUTH_BASE_URL}?{auth_query}'

# WebDriverの設定
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

# LINE WORKSログインページにアクセス
driver.get(auth_url)

# ユーザーIDとパスワードの入力
driver.find_element(By.ID, "inputId").send_keys(LOGIN_ID)
driver.find_element(By.ID, "password").send_keys(PASSWORD)

# ログイン処理の実行
driver.execute_script("login()")

# リダイレクトを待つ
time.sleep(5)

# アラートが表示されているかを確認
try:
    alert = driver.switch_to.alert
    alert_text = alert.text
    alert.accept()
    print(f"Alert detected: {alert_text}")

except NoAlertPresentException:
    pass

# リダイレクト後のURLからAuthorization Codeを取得
auth_code = driver.current_url.split('code=')[1].split('&')[0]

# WebDriverを終了
driver.quit()

# Access Tokenの取得
token_url = 'https://auth.worksmobile.com/oauth2/v2.0/token'
data = {
    'code': auth_code,
    'grant_type': 'authorization_code',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'redirect_uri': REDIRECT_URI
}
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json().get('access_token')
    print("Access Token:", access_token)
else:
    print("Error fetching Access Token:")
    print(f"HTTP Status Code: {response.status_code}")
    print(f"Response: {response.json()}")